In [1]:
%AddJar file:///home/testuser/GPUEnabler/Resources/gpu-enabler_2.11-1.0.0.jar
%AddJar file:///home/testuser/GPUEnabler/Resources/gpu-enabler-examples_2.11-1.0.0.jar

Starting download from file:///home/testuser/GPUEnabler/Resources/gpu-enabler_2.11-1.0.0.jar
Finished download of gpu-enabler_2.11-1.0.0.jar
Starting download from file:///home/testuser/GPUEnabler/Resources/gpu-enabler-examples_2.11-1.0.0.jar
Finished download of gpu-enabler-examples_2.11-1.0.0.jar


In [1]:
import java.util.Random
import com.ibm.gpuenabler.CUDADSImplicits._
import com.ibm.gpuenabler._
import org.apache.spark.sql.{Dataset, DataFrame, SparkSession}
import scala.math._
import scala.collection.mutable
import org.apache.spark.sql.functions._

In [2]:
spark.conf.get("spark.executor.memory")

40g

In [3]:
//spark.conf.set("spark.executor.memory", "20g")
//spark.conf.set("spark.driver.memory", "20g")

Name: Syntax Error.
Message: 
StackTrace: 

In [4]:
case class DataPointKMeans(features: Array[Double])
case class ClusterIndexes(features: Array[Double], index: Int)
case class Results(s0: Array[Int], s1: Array[Double], s2: Array[Double])

defined class DataPointKMeans
defined class ClusterIndexes
defined class Results


In [5]:
//val sqlC = new org.apache.spark.sql.SQLContext(sc)
//import sqlC.implicits._

val sparkSession = SparkSession.builder.getOrCreate()
import sparkSession.implicits._


def getDataSet(spark: SparkSession, inputPath: String): Dataset[DataPointKMeans] = {

val sparkSession = SparkSession.builder.getOrCreate()
import sparkSession.implicits._
    
    val rawinputDF = spark.read.option("header", "false").option("inferSchema", "true").csv(inputPath)

    val pointsCached = rawinputDF.map(x=> {
      val rowElem = x.getString(0).trim.split(" ")
      val len = rowElem.length
      val buffer = new mutable.ListBuffer[Double]()
      (0 until len).foreach { idx =>
        if(!rowElem(idx).isEmpty)
          buffer += rowElem(idx).toDouble
      }

      DataPointKMeans(buffer.toArray)
    })

    pointsCached
  }

sparkSession = org.apache.spark.sql.SparkSession@358a7c68


getDataSet: (spark: org.apache.spark.sql.SparkSession, inputPath: String)org.apache.spark.sql.Dataset[DataPointKMeans]


Spark local-1500043706446: Some(http://172.17.0.2:4041)

In [13]:
val sparkSession = SparkSession.builder.getOrCreate()
import sparkSession.implicits._

def getDataFrame(spark: SparkSession, path: String): DataFrame = {
 // import spark.implicits._

    val rawinputDF: DataFrame = spark.read
      .option("header", "false")
      .option("inferSchema", "true")
      .csv(path)

    rawinputDF
}
  
  val pointsCached = getDataFrame(spark, "./data/rawimageX3").cache


  
  val toDouble = udf[Array[Double], String]( x => {
      val rowElem = x.split(" ")
      val len = rowElem.length
      val buffer = new mutable.ListBuffer[Double]()
      (0 until len). foreach { idx =>
        if(!rowElem(idx).isEmpty)
          buffer += rowElem(idx).toDouble
      }

      (buffer.toArray)})
      
  // val schema = StructType( StructField(fieldName, StringType, nullable = true))
  val data1 = pointsCached.toDF("Strfeatures")
  data1.printSchema


  val data = data1.withColumn("features", toDouble(data1("Strfeatures"))).select("features")// .as[DataPointKMeans]
  data.printSchema
  println(data.count())
  // println(data.head.features.length)
val means = data.rdd
println(means.count())

//data.cacheGpu(true)
//data.loadGpu()
  

root
 |-- Strfeatures: string (nullable = true)

root
 |-- features: array (nullable = true)
 |    |-- element: double (containsNull = false)

2351880
2351880                                                                         


Name: Syntax Error.
Message: 
StackTrace: 

In [9]:
val means = data.rdd
println(means.count())

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 17 in stage 10.0 failed 1 times, most recent failure: Lost task 17.0 in stage 10.0 (TID 107, localhost, executor driver): java.lang.ClassCastException: $line19.$read$$iw$$iw cannot be cast to $line19.$read$$iw$$iw
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificSafeProjection.apply(Unknown Source)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.util.Utils$.getIteratorSize(Utils.scala:1762)
	at org.apache.spark.rdd.RDD$$anonfun$count$1.apply(RDD.scala:1158)
	at org.apache.spark.rdd.RDD$$anonfun$count$1.apply(RDD.scala:1158)
	at org.apache.spark.SparkContext$$anonfun$runJob$5.apply(SparkContext.scala:1951)
	at org.apache.spark.SparkContext$$anonfun$runJob$5.apply(SparkContext.scala:1951)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:87)
	at org.apache.spark.sched

In [7]:
val inputPath="./data/rawimageX3"
val k = 16
val iter = 10

inputPath = ./data/rawimageX3
k = 16
iter = 10


10

In [8]:
val pointsCached = getDataSet(spark, inputPath).cache
val data = pointsCached
val N = pointsCached.count()
val d = data.head.features.length

[Stage 2:==================================>                     (16 + 10) / 26]

pointsCached = [features: array<double>]
data = [features: array<double>]
N = 2351880
d = 3


3

In [18]:
val pointsCached = getDataFrame(spark, inputPath).cache
val data = pointsCached
val N = pointsCached.count()
//val d = data.head.features.length

Name: Syntax Error.
Message: 
StackTrace: 

In [18]:
val rawinputDF = spark.read.option("header", "false").option("inferSchema", "true").csv(inputPath)


In [19]:

val sqlC = new org.apache.spark.sql.SQLContext(sc)
import sqlC.implicits._

val pointsCached = rawinputDF.map(in => {
    val buffer = new mutable.ListBuffer[Double]()
    in.getString(0).trim.split(" ").foreach(x=>if (!x.isEmpty) buffer += x.toDouble)
    (buffer.toArray)
}).cache



In [20]:
val data = pointsCached.toDF("features").as[DataPointKMeans]
val N = pointsCached.count()
val d = data.head.features.length


In [21]:
println(s"d: ${d} ; N : ${N}; k : ${k}")

d: 3 ; N : 2351880; k : 16


In [9]:
val epsilon = 0.5
var changed = true
var iteration = 0
var cost = 0.0
val ptxURL = "/data/SparkGPUKmeans.ptx"

epsilon = 0.5
changed = true
iteration = 0
cost = 0.0
ptxURL = /data/SparkGPUKmeans.ptx


/data/SparkGPUKmeans.ptx

In [10]:
    val centroidFn = DSCUDAFunction(
        "getClusterCentroids",
        Array("features"),
        Array("index"),
        ptxURL)

centroidFn = DSCUDAFunction(getClusterCentroids,WrappedArray(features),WrappedArray(index),/data/SparkGPUKmeans.ptx,None,None,None)


DSCUDAFunction(getClusterCentroids,WrappedArray(features),WrappedArray(index),/data/SparkGPUKmeans.ptx,None,None,None)

In [11]:
val dimensions1 = (size: Long, stage: Int) => stage match {
  case 0 => (450, 3, 1, 16, 1, 1)
}

val gpuParams1 = gpuParameters(dimensions1)

val interFn = DSCUDAFunction(
    "calculateIntermediates",
    Array("features", "index"),
    Array("s0", "s1", "s2"),
    ptxURL,
    Some((size: Long) => 1),
    Some(gpuParams1), outputSize=Some(450))

dimensions1 = > (Int, Int, Int, Int, Int, Int) = <function2>
gpuParams1 = gpuParameters(<function2>,None)
interFn = DSCUDAFunction(calculateIntermediates,WrappedArray(features, index),WrappedArray(s0, s1, s2),/data/SparkGPUKmeans.ptx,Some(<function1>),Some(gpuParameters(<function2>,None)),Some(450))


DSCUDAFunction(calculateIntermediates,WrappedArray(features, index),WrappedArray(s0, s1, s2),/data/SparkGPUKmeans.ptx,Some(<function1>),Some(gpuParameters(<function2>,None)),Some(450))

In [12]:
val dimensions2 = (size: Long, stage: Int) => stage match {
      case 0 => (1, 3, 1, 16, 1, 1)
    }

val gpuParams2 = gpuParameters(dimensions2)

val sumFn = DSCUDAFunction(
    "calculateFinal",
    Array("s0", "s1", "s2"),
    Array("s0", "s1", "s2"),
    ptxURL,
    Some((size: Long) => 1),
    Some(gpuParams2), outputSize=Some(1))

dimensions2 = > (Int, Int, Int, Int, Int, Int) = <function2>
gpuParams2 = gpuParameters(<function2>,None)
sumFn = DSCUDAFunction(calculateFinal,WrappedArray(s0, s1, s2),WrappedArray(s0, s1, s2),/data/SparkGPUKmeans.ptx,Some(<function1>),Some(gpuParameters(<function2>,None)),Some(1))


DSCUDAFunction(calculateFinal,WrappedArray(s0, s1, s2),WrappedArray(s0, s1, s2),/data/SparkGPUKmeans.ptx,Some(<function1>),Some(gpuParameters(<function2>,None)),Some(1))

In [13]:
def addArr(lhs: Array[Double], rhs: Array[Double]) = {
require(lhs.length == rhs.length, "equal lengths")

lhs.zip(rhs).map { case (x, y) => x + y }
}

def addArrI(lhs: Array[Int], rhs: Array[Int]) = {
require(lhs.length == rhs.length, "equal lengths")

lhs.zip(rhs).map { case (x, y) => x + y }
}
def func1(p: DataPointKMeans): ClusterIndexes = {
  ClusterIndexes(p.features, 0)
}

def func2(c: ClusterIndexes): Results = {
  Results(Array.empty, Array.empty, Array.empty)
}

def func3(r1: Results, r2: Results): Results = {
  Results(addArrI(r1.s0, r2.s0),
    addArr(r1.s1, r2.s1),addArr(r1.s2, r2.s2))
}

addArr: (lhs: Array[Double], rhs: Array[Double])Array[Double]
addArrI: (lhs: Array[Int], rhs: Array[Int])Array[Int]
func1: (p: DataPointKMeans)ClusterIndexes
func2: (c: ClusterIndexes)Results
func3: (r1: Results, r2: Results)Results


In [14]:
val sparkSession = SparkSession.builder.getOrCreate()
import sparkSession.implicits._
data.cacheGpu(true)
data.loadGpu()

[Stage 5:>                                                        (0 + 26) / 26]

sparkSession = org.apache.spark.sql.SparkSession@358a7c68


2351880

In [15]:
val means = data.rdd.takeSample(true, k, 42)

means = Array(DataPointKMeans([D@1e4d778e), DataPointKMeans([D@6fb525c4), DataPointKMeans([D@2fac7f94), DataPointKMeans([D@2cea7dfb), DataPointKMeans([D@59ee9eef), DataPointKMeans([D@1fb30b13), DataPointKMeans([D@532f97ae), DataPointKMeans([D@2cd53d95), DataPointKMeans([D@1e1d706), DataPointKMeans([D@60dbfba), DataPointKMeans([D@29b0da32), DataPointKMeans([D@2c8c6304), DataPointKMeans([D@235ac874), DataPointKMeans([D@484153ca), DataPointKMeans([D@670f61ba), DataPointKMeans([D@2854aa1c))


[DataPointKMeans([D@1e4d778e), DataPointKMeans([D@6fb525c4), DataPointKMeans([D@2fac7f94), DataPointKMeans([D@2cea7dfb), DataPointKMeans([D@59ee9eef), DataPointKMeans([D@1fb30b13), DataPointKMeans([D@532f97ae), DataPointKMeans([D@2cd53d95), DataPointKMeans([D@1e1d706), DataPointKMeans([D@60dbfba), DataPointKMeans([D@29b0da32), DataPointKMeans([D@2c8c6304), DataPointKMeans([D@235ac874), DataPointKMeans([D@484153ca), DataPointKMeans([D@670f61ba), DataPointKMeans([D@2854aa1c)]